In [ ]:
# %pip install langchain-openai python-dotenv
# %pip install langchain-core langchain-upstage
# %pip install --upgrade --quiet  docx2txt
# %pip install langchain_community
# %pip install -qU langchain-text-splitters
%pip install langchain-chroma

## 주피터 노트북 팁
- 노트 실행 에러시 노트 껏다 켜서 다시해보기

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 모델 연결

- 한국말은 임베딩 모델 Upstage AI(SolarLLM) 사용하는게 벡터간 유사도 성능 좋음

- 업스테이지 가입 → 무료 30달러 크레딧 → api 키 받기 → 예시 코드 작성

[랭체인 문서 링크](https://python.langchain.com/v0.2/docs/integrations/providers/upstage/)

In [2]:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI()
# llm.invoke("한국의 수도는?")

from langchain_upstage import ChatUpstage

chat = ChatUpstage()

In [3]:
response = chat.invoke("한국의 수도는?")
print(response)

content='대한민국의 수도는 서울입니다. 서울은 한국에서 가장 큰 도시이기도 하며, 정치, 경제, 문화, 교통의 중심지입니다. 서울은 한강을 중심으로 한 도시로, 다양한 역사적인 유적지와 현대적인 건축물, 맛있는 음식, 활발한 밤문화 등으로 유명합니다. 서울은 세계적인 도시 중 하나로 꼽히며, 많은 관광객들이 방문하는 인기 있는 도시입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 15, 'total_tokens': 93, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'solar-mini-240612', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-0615670f-91b0-4c4b-95c6-a3b1b4820cc9-0' usage_metadata={'input_tokens': 15, 'output_tokens': 78, 'total_tokens': 93, 'input_token_details': {}, 'output_token_details': {}}


## 문서 로드


[소득세법 문서 출처](https://law.go.kr/%EB%B2%95%EB%A0%B9/%EC%86%8C%EB%93%9D%EC%84%B8%EB%B2%95)

- 자료는 워드로 다운 받는게 좋음.
- pdf는 줄바꿈시 한글 단어가 짤릴 수 있음
- 워드 문서 다운 받은 후 확장자를 doc에서 docx로 변경 필요

[랭체인 문서](https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/#using-docx2txt)

In [ ]:
# 문서 로드
from langchain_community.document_loaders import Docx2txtLoader, TextLoader

loader = TextLoader("./docs/dorm.md")
data = loader.load()
data

[Document(metadata={'source': './dorm.md'}, page_content='# SK 하이닉스 이천 기숙사\n\n## 소개\n\n이천 기숙사는 경기도 이천시 부발읍 일대 44만 평 부지에 4개 Site, 17개동, 3,200여 개 호실을 갖추고 여러분을 기다리고 있습니다.\n기숙사와 함께 다양한 커뮤니티 시설이 제공되며 라운지, 카페, 헬스장, 문화 강좌실 등 생활지원시설 및 문화시설에서 입숙인들이 안전하고 편안한 시간을 즐길 수 있도록 다양한 지원과 서비스를 준비하고 있습니다.\n\n## 기숙사동\n\n1. 행복1마을\n\n   - 주소: 경기도 이천시 대산로 288번길, 89\n\n2. 행복2마을\n\n   - 주소: 경기도 이천시 대월면 대산로 288번길, 48\n\n3. 청운기숙사\n\n   - 주소: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 청운관\n\n4. 현우기숙사\n\n   - 주소: 경기도 이천시 부발읍 경충대로 2041번길, 54\n\n## 기숙사 홈페이지\n\n- 홈페이지 이름: 하이홈(Hihome)\n- 주소: https://hihome.skhynix.com/main\n- 설명: 구성원의 생활/지원 서비스(입·퇴실, 이동, 하자 등)는 더욱 빠르게 해결하고, 주거 시설 Infra (Cafétreia, 주차, 택배 등) 및 문화/편의시설 운영 정보도 실시간으로 제공해 더욱 편리한 생활을 지원합니다\n\n## 관리지원 센터 및 시설 연락처와 운영정보\n\n- 현우기숙사 생활지원센터\n\n* 연락처: 031-5185-3420\n* 위치: 경기도 이천시 부발읍 경중대로 2041번길 54 현우기숙사\n* 운영시간: 평일 08:30~17:30 / 유연근무\n\n- 행복마을 시설관리\n\n* 연락처: 031-5185-3883\n* 위치: [ 행복 1마을 ]경기도 이천시 대산로 288번길, 89 [ 행복 2마을 ] 경기도 이천시 대산로 288번길, 48\n* 운영시간: 평일 08:30~17:30 / 유연근무\n\n-

## 텍스트 쪼개기

- 텍스틑 쪼개는 방법은 보통 recursive 또는 character를 사용함.
- recursive가 구분자 설정을 할 수 있어서 성능이 더 좋음

[랭체인 문서](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)

In [5]:
# 문서 쪼개기
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1500,
    chunk_overlap=200,
    # length_function=len,
    # is_separator_regex=False,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

In [6]:
# len(document_list)
# print(type(document_list))
# print(type(document_list[0]))
# print(document_list[0])
document_list[0]

Document(metadata={'source': './dorm.md'}, page_content='# SK 하이닉스 이천 기숙사\n\n## 소개\n\n이천 기숙사는 경기도 이천시 부발읍 일대 44만 평 부지에 4개 Site, 17개동, 3,200여 개 호실을 갖추고 여러분을 기다리고 있습니다.\n기숙사와 함께 다양한 커뮤니티 시설이 제공되며 라운지, 카페, 헬스장, 문화 강좌실 등 생활지원시설 및 문화시설에서 입숙인들이 안전하고 편안한 시간을 즐길 수 있도록 다양한 지원과 서비스를 준비하고 있습니다.\n\n## 기숙사동\n\n1. 행복1마을\n\n   - 주소: 경기도 이천시 대산로 288번길, 89\n\n2. 행복2마을\n\n   - 주소: 경기도 이천시 대월면 대산로 288번길, 48\n\n3. 청운기숙사\n\n   - 주소: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 청운관\n\n4. 현우기숙사\n\n   - 주소: 경기도 이천시 부발읍 경충대로 2041번길, 54\n\n## 기숙사 홈페이지\n\n- 홈페이지 이름: 하이홈(Hihome)\n- 주소: https://hihome.skhynix.com/main\n- 설명: 구성원의 생활/지원 서비스(입·퇴실, 이동, 하자 등)는 더욱 빠르게 해결하고, 주거 시설 Infra (Cafétreia, 주차, 택배 등) 및 문화/편의시설 운영 정보도 실시간으로 제공해 더욱 편리한 생활을 지원합니다\n\n## 관리지원 센터 및 시설 연락처와 운영정보\n\n- 현우기숙사 생활지원센터\n\n* 연락처: 031-5185-3420\n* 위치: 경기도 이천시 부발읍 경중대로 2041번길 54 현우기숙사\n* 운영시간: 평일 08:30~17:30 / 유연근무\n\n- 행복마을 시설관리\n\n* 연락처: 031-5185-3883\n* 위치: [ 행복 1마을 ]경기도 이천시 대산로 288번길, 89 [ 행복 2마을 ] 경기도 이천시 대산로 288번길, 48\n* 운영시간: 평일 08:30~17:30 / 유연근무\n\n- 

# 임베딩

[업스테이지 문서](https://console.upstage.ai/docs/capabilities/embeddings)

[크로마 랭체인 문서]()

In [3]:
# 임베딩

# from langchain_openai import AzureOpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings


embedding = UpstageEmbeddings(
    model="embedding-query"
)

In [ ]:
from langchain_chroma import Chroma

database = Chroma(collection_name="dorm",
    embedding_function=embedding)
database.add_documents(documents=document_list)



In [ ]:
embedding.aembed_documents

In [4]:
from langchain_chroma import Chroma

# # 인메모리가 아닌 sqlite로 저장하는 방법
# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="chroma-tax", persist_directory="./chroma")

# 저장한 크로마 불러오는 방법
database = Chroma(collection_name="dorm", persist_directory="./chroma_db", embedding_function=embedding)

In [9]:
database.add_documents(documents=document_list)

['a2460edc-d58b-4d0d-a3a6-afaa7684bd66',
 'c9fc58ec-410b-401a-a6c1-c6fad968c2a3',
 '72c2f030-991b-4b87-89e8-2b7512cb50dc']

In [5]:
# 존대말로 하면 답변을 잘해줌
# 영어에서는 please
query = '행복2마을 헬스장 운영시간은?'
# 유사도 검색
retrieved_docs = database.similarity_search(query, k=4)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


In [ ]:
retrieved_docs


# Chat
- OpenAI  
[랭체인 문서](https://python.langchain.com/v0.2/docs/integrations/chat/openai/#integration-details)

- Upstage  
[랭체인 문서](https://python.langchain.com/docs/integrations/chat/upstage/)

In [12]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [12]:
# 페르소나 주입
prompt = f"""[Identity]
- [Context]를 참고해서 [Questions]에 답변해주세요

[Context]
{retrieved_docs}

[Questions]
{query}
"""


In [13]:
ai_message  =llm.invoke(prompt)

In [14]:
ai_message.content


'연봉이 5천만원인 직장인의 경우, 소득세는 다음과 같이 계산됩니다.\n\n소득세율은 누진세율로 적용되며, 소득이 높아질수록 세율도 높아집니다. 따라서, 5천만원인 경우의 소득세는 다음과 같이 계산됩니다.\n\n1. 총 급여액에서 근로소득공제액을 뺍니다.\n2. 근로소득공제액을 뺀 금액에 대한 소득세를 계산합니다.\n3. 계산된 소득세에서 추가 공제액(예: 인적공제, 연금공제, 의료비공제 등)을 뺍니다.\n4. 최종적으로 납부해야 할 소득세를 계산합니다.\n\n연봉이 5천만원인 경우, 소득세는 1,962,000원이 됩니다.'

# Langchain을 활용해서 매번 세팅 안해주는법

## LangchainQA
- deprecated 됨
- __create_retrieval_chain__ 이용
  
[Migration 문서](https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/)
  
[create_retrieval_chain 문서](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval.create_retrieval_chain.html#langchain-chains-retrieval-create-retrieval-chain)

In [19]:
# 저장한 크로마 불러오는 방법
database = Chroma(collection_name="dorm", persist_directory="./chroma_db", embedding_function=embedding)

query = '세탁소 연락처는?'
llm = ChatUpstage()




In [20]:
from langchain import hub

# Lagacy
# prompt = hub.pull("rlm/rag-prompt")

# qa_chain = RetrievalQA.from_llm(
#     llm, 
#     retriever=database.as_retriever(),
#     chain_type_kwargs={"prompt": prompt}
# )

# ai_message = qa_chain({"query": query})

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
retrieval_qa_chat_prompt = prompt = hub.pull("teddynote/rag-prompt-korean")

combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(database.as_retriever(), combine_docs_chain)

ai_message = rag_chain.invoke({"question": query, "input": query})





/Users/boosj/.pyenv/versions/3.10.15/envs/llm/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


In [ ]:
ai_message